<a href="https://colab.research.google.com/github/WhiteFox-Lugh/Pratice-TextAnalysis/blob/master/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

逐次実行していけば所定の処理ができるようにしてあります。
（ただし、外部テキストファイルなどは適切な場所に配置してください）

In [1]:
!pip install nltk
!pip install gensim

In [37]:
import nltk
import numpy as np
import pandas as pd
import re
from google.colab import drive
from nltk.corpus import wordnet as wn

In [23]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

text の引用元

全て最初の概要部分です（注釈番号は抜いています）

* text1.txt  : https://en.wikipedia.org/wiki/Canis
* text2.txt  : https://en.wikipedia.org/wiki/Wolf
* text3.txt  : https://en.wikipedia.org/wiki/Coyote
* text4.txt  : https://en.wikipedia.org/wiki/African_golden_wolf
* text5.txt  : https://en.wikipedia.org/wiki/Dhole
* text6.txt  : https://en.wikipedia.org/wiki/African_wild_dog
* text7.txt  : https://en.wikipedia.org/wiki/Red_fox
* text8.txt  : https://en.wikipedia.org/wiki/Arctic_fox
* text9.txt  : https://en.wikipedia.org/wiki/Fennec_fox
* text10.txt : https://en.wikipedia.org/wiki/Kitsune

In [9]:
# drive のマウント（colaboratory 用）
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [38]:
# テキスト読み込み
# テキストデータは適切なところに配置してください
text_name = ['canis', 'wolf', 'coyote', 'african_golden_wolf', 'dhole', 'african_wild_dog', 'red_fox', 'arctic_fox', 'fennec_fox', 'kitsune']
text_correction = {}
for i in range(len(text_name)):
  txt = f = open('drive/My Drive/Colab Notebooks/text' + str(i+1) + '.txt')
  data = f.read()
  name = text_name[i]
  text_correction[name] = data

In [39]:
# 読み込み確認
for txt in text_correction.values():
  print(txt)

Canis is a genus of the Caninae containing multiple extant species, such as wolves, dogs, coyotes and jackals. Species of this genus are distinguished by their moderate to large size, their massive, well-developed skulls and dentition, long legs, and comparatively short ears and tails.
The wolf (Canis lupus), also known as the gray wolf or grey wolf, is a large canine native to Eurasia and North America. More than thirty subspecies of Canis lupus have been recognized, and gray wolves, as colloquially understood, comprise non-domestic/feral subspecies. The wolf is the largest extant member of Canidae, males averaging 40 kg (88 lb) and females 37 kg (82 lb). Wolves measure 105–160 cm (41–63 in) in length and 80–85 cm (31–33 in) at shoulder height. The wolf is also distinguished from other Canis species by its less pointed ears and muzzle, as well as a shorter torso and a longer tail. The wolf is nonetheless related closely enough to smaller Canis species, such as the coyote and the golde

In [44]:
# preprocessing
# tokenize
text_tokenized = {}

def tokenize_text(text):
  text = re.sub('[.,()\[\]]', '', text)
  ret = text.split()
  return ret

for i in range(len(text_name)):
  name = text_name[i]
  txt = text_correction[name]
  text_tokenized[name] = tokenize_text(txt)

In [45]:
# tokenize 確認
for txt in text_tokenized.values():
  print(txt)

['Canis', 'is', 'a', 'genus', 'of', 'the', 'Caninae', 'containing', 'multiple', 'extant', 'species', 'such', 'as', 'wolves', 'dogs', 'coyotes', 'and', 'jackals', 'Species', 'of', 'this', 'genus', 'are', 'distinguished', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'well-developed', 'skulls', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ears', 'and', 'tails']
['The', 'wolf', 'Canis', 'lupus', 'also', 'known', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'is', 'a', 'large', 'canine', 'native', 'to', 'Eurasia', 'and', 'North', 'America', 'More', 'than', 'thirty', 'subspecies', 'of', 'Canis', 'lupus', 'have', 'been', 'recognized', 'and', 'gray', 'wolves', 'as', 'colloquially', 'understood', 'comprise', 'non-domestic/feral', 'subspecies', 'The', 'wolf', 'is', 'the', 'largest', 'extant', 'member', 'of', 'Canidae', 'males', 'averaging', '40', 'kg', '88', 'lb', 'and', 'females', '37', 'kg', '82', 'lb', 'Wolves', 'measure', '105–160', 'cm'

In [47]:
# lemmatize
def lemmatize_word(word_set):
  ret = []
  for word in word_set:
    word=word.lower()
    lemma = wn.morphy(word)
    if lemma is None:
      ret.append(word)
    else:
      ret.append(lemma)
  return ret

for i in range(len(text_name)):
  name = text_name[i]
  word_set = text_tokenized[name]
  print(lemmatize_word(word_set))

['canis', 'be', 'a', 'genus', 'of', 'the', 'caninae', 'contain', 'multiple', 'extant', 'species', 'such', 'as', 'wolf', 'dog', 'coyote', 'and', 'jackal', 'species', 'of', 'this', 'genus', 'are', 'distinguish', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'well-developed', 'skull', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ear', 'and', 'tails']
['the', 'wolf', 'canis', 'lupus', 'also', 'know', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'be', 'a', 'large', 'canine', 'native', 'to', 'eurasia', 'and', 'north', 'america', 'more', 'than', 'thirty', 'subspecies', 'of', 'canis', 'lupus', 'have', 'be', 'recognize', 'and', 'gray', 'wolf', 'as', 'colloquially', 'understand', 'comprise', 'non-domestic/feral', 'subspecies', 'the', 'wolf', 'be', 'the', 'large', 'extant', 'member', 'of', 'canidae', 'male', 'average', '40', 'kg', '88', 'lb', 'and', 'female', '37', 'kg', '82', 'lb', 'wolf', 'measure', '105–160', 'cm', '41–63', 'in', 'in', 'le